In [41]:
import dlib
from skimage import io
from imageio import imread
from scipy.spatial import distance, KDTree
import json

In [42]:
sp = dlib.shape_predictor('../models/shape_predictor_68_face_landmarks.dat')
facerec = dlib.face_recognition_model_v1('../models/dlib_face_recognition_resnet_model_v1.dat')
detector = dlib.get_frontal_face_detector()

class FaceDetecting:

    descriptor = None
    
    def __init__(self, person_name: str, person_image: str, person_info=''):
        self.name = person_name
        self.image = person_image
        self.info = person_info
        self.descriptor = self.find_descriptor()
    
    def find_descriptor(self):
        dets = detector(self.image, 1)
        
        for k, d in enumerate(dets):
            shape = sp(self.image, d)
        
        return facerec.compute_face_descriptor(self.image, shape)
            

def euclidian_distance(vector1, vector2):
    return distance.euclidean(vector1, vector2)
    

In [43]:
import urllib
from bs4 import BeautifulSoup as bs
import re

hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

In [32]:
actors_array = []

def get_actors_from_page(link: str, from_page: int):
    actors = bs(urllib.request.urlopen(link + str(from_page)).read()).find("div", {"class": "lister-list"}).find_all("div", {"class": "lister-item mode-detail"})
    for actor in actors:
        print(actor.find('img')['src'])
        try:
            actors_array.append(FaceDetecting(person_name=actor.find('h3').find('a').text, person_image=imread(actor.find('img')['src'])))
        except:
            print('Wrong image format for :' + actor.find('h3').find('a').text)
    

for i in range(10):
    print(i)
    get_actors_from_page('https://www.imdb.com/search/name/?gender=male,female&ref_=rlm&start=', 1 + 50 * i)

0
https://m.media-amazon.com/images/M/MV5BYWZmMDFlMDYtYzk3My00YTg1LTllZWUtYzY2Y2QyNjk2NTZmXkEyXkFqcGdeQXVyMzcxMTAwMDM@._V1_UY209_CR52,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTUxNTM4OTgwN15BMl5BanBnXkFtZTcwODMzOTQ2OQ@@._V1_UY209_CR10,0,140,209_AL_.jpg
Wrong image format for : Erin Moriarty

https://m.media-amazon.com/images/M/MV5BMTgyMjI3ODA3Nl5BMl5BanBnXkFtZTcwNzY2MDYxOQ@@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTgxNDcwMzU2Nl5BMl5BanBnXkFtZTcwNDc4NzkzOQ@@._V1_UY209_CR8,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMzQ0NDg0MjQ3NV5BMl5BanBnXkFtZTgwOTI5OTAwODE@._V1_UY209_CR8,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BYjgzMmQ2ZDctOTc0Ny00NDYzLWIwMTMtNzY0MDE2ZGE1Y2NlXkEyXkFqcGdeQXVyMTEyNDk1MjY@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTU2Njk0NTUyNl5BMl5BanBnXkFtZTcwODE0OTgyNw@@._V1_UY209_CR9,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTUzMDM1NzkxN15BMl5BanBnXkFtZTgwO

https://m.media-amazon.com/images/M/MV5BMTYzODU2OTI2Nl5BMl5BanBnXkFtZTgwMTUyMTAwMzE@._V1_UY209_CR5,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BOGYzNjE2MDYtMjhhNi00MGE4LTk1ZDUtM2ZmODZlMmQ2ZjE3XkEyXkFqcGdeQXVyNjUxMjc1OTM@._V1_UY209_CR87,0,140,209_AL_.jpg
Wrong image format for : Maya Hawke

https://m.media-amazon.com/images/M/MV5BZWRiOTdjN2UtY2M1Yy00YzBjLWJmMzQtYTAyZGY4ZGFmZDRkXkEyXkFqcGdeQXVyNDI3NzU1NDE@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTc1NTc2NDI4MF5BMl5BanBnXkFtZTgwOTc5NDE5OTE@._V1_UY209_CR34,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMzI5NDIzNTQ1Nl5BMl5BanBnXkFtZTgwMjQ0Mzc1MTE@._V1_UY209_CR3,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BNDZiZjg2MDktOWU5ZC00NzVjLTg5ZTUtMGU4MjA3ZmZmNmU2XkEyXkFqcGdeQXVyNzQxMDc5MzE@._V1_UY209_CR60,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTEzNjU3OTQzMjBeQTJeQWpwZ15BbWU4MDE3ODk5NDkx._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BYTUxO

Wrong image format for : Annabelle Wallis

https://m.media-amazon.com/images/M/MV5BMzk4MDIzNjcwNV5BMl5BanBnXkFtZTgwMTIyMjgwNDI@._V1_UY209_CR13,0,140,209_AL_.jpg
Wrong image format for : Taika Waititi

https://m.media-amazon.com/images/M/MV5BMTc1NDcwMDI5MV5BMl5BanBnXkFtZTcwMzI4OTIzMQ@@._V1_UX140_CR0,0,140,209_AL_.jpg
Wrong image format for : Michael Cerveris

https://m.media-amazon.com/images/M/MV5BMTc4MTAxMDMxMV5BMl5BanBnXkFtZTgwMTYxNjE5MDI@._V1_UY209_CR8,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTM0ODU5Nzk2OV5BMl5BanBnXkFtZTcwMzI2ODgyNQ@@._V1_UY209_CR3,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMjE1MDkxMjQ3NV5BMl5BanBnXkFtZTcwMzQ3Mjc4MQ@@._V1_UY209_CR6,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BNjcwNzg4MjktNDNlMC00M2U1LWJmMjgtZTVkMmI4MDI2MTVmXkEyXkFqcGdeQXVyMjI4MDI0NTM@._V1_UY209_CR8,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTgxMjM2NTAyMV5BMl5BanBnXkFtZTgwOTU4NjU2NDE@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon

https://m.media-amazon.com/images/M/MV5BMzI3NTYwMzIxM15BMl5BanBnXkFtZTcwMzI1ODY1NA@@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BZGUxMDNkMDAtYmU4NS00MDNkLTgyYTgtYTY0NjEzOGMzYmQwXkEyXkFqcGdeQXVyMjU3OTMyODI@._V1_UY209_CR72,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTQ3NzM3MTc2NF5BMl5BanBnXkFtZTcwODMxNjA0NA@@._V1_UY209_CR7,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTU5Mjg5Mjg1MF5BMl5BanBnXkFtZTcwNzEzMzY0NA@@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BZmRhZWVjZWMtMzUwZi00OTI3LThiMzAtY2VmMjU5YjQ0YjVjXkEyXkFqcGdeQXVyODMzMTM1MDc@._V1_UY209_CR11,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTU5NjEwOTgwMF5BMl5BanBnXkFtZTgwOTEzMDk1NTM@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BNTMwMDg1MDkyNV5BMl5BanBnXkFtZTcwNzEwMzg1Nw@@._V1_UY209_CR87,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTUwNjQ0ODkxN15BMl5BanBnXkFtZTcwMDc5NjQwNw@@._V1_UY209_CR7,0,140,209_AL_.j

https://m.media-amazon.com/images/M/MV5BMTkxMzk4MjQ4MF5BMl5BanBnXkFtZTcwMzExODQxOA@@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMmI2NTU4NTYtNTdiZi00YjNhLTk5ZGUtMmFkZjBjYTliYjg1XkEyXkFqcGdeQXVyNzczNzg0ODc@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BZWQyY2UwNmItMTI5Yy00YTAzLThjODAtMmYyODdmZWYxYmM5XkEyXkFqcGdeQXVyNjA0OTkxNjQ@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMjM4Nzc3Njc0N15BMl5BanBnXkFtZTgwMjE0MDEzMTE@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTM3NzQ4ODU0M15BMl5BanBnXkFtZTcwMDM4Njg3OA@@._V1_UY209_CR8,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BYjAxMDgyYWEtOTRmNS00MjFiLTg5MmMtNmFkZDVhNjI0ZmJiXkEyXkFqcGdeQXVyMTM5NzkxODQ@._V1_UY209_CR8,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BZjAwODgwNTItZjNmZS00MDdhLWJhNGQtZDkxYjgyMjU2YzNiXkEyXkFqcGdeQXVyMzEwNzc0Mg@@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTYwNjQ5MTI1N

https://m.media-amazon.com/images/M/MV5BMjExNjY5NDY0MV5BMl5BanBnXkFtZTgwNjQ1Mjg1MTI@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTQ3ODE2NTMxMV5BMl5BanBnXkFtZTgwOTIzOTQzMjE@._V1_UY209_CR14,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BOTM5NzI1NTMwN15BMl5BanBnXkFtZTcwOTQ0NjExNw@@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMjM4MTg3NjM5M15BMl5BanBnXkFtZTgwNjkzMTA2MDI@._V1_UY209_CR3,0,140,209_AL_.jpg
Wrong image format for : Bojana Novakovic

https://m.media-amazon.com/images/M/MV5BMDM5YjI2NDItYWY1My00ZmUyLTgyZjMtOGQzMmEyNjk0ODU5XkEyXkFqcGdeQXVyMjQwMDg0Ng@@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BZWE5OGY0ZjYtZGExMi00ZDc3LWI4OGYtNzdkMTg0NWNkMzJlXkEyXkFqcGdeQXVyMjQwMDg0Ng@@._V1_UY209_CR80,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTc2MDMzODU0OV5BMl5BanBnXkFtZTgwODIwOTAxMzI@._V1_UY209_CR21,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BNzZhMjYzMGEtYTA2Zi00ODI1LTgyMzI

https://m.media-amazon.com/images/M/MV5BMjA1MTQ3NzU1MV5BMl5BanBnXkFtZTgwMDE3Mjg0MzE@._V1_UY209_CR35,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMjEzMjk4NDU4MF5BMl5BanBnXkFtZTcwMDMyNjQzMg@@._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTQzMTcxMzcwN15BMl5BanBnXkFtZTYwNjIwMjc1._V1_UY209_CR4,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTY1OTE4NzQwMl5BMl5BanBnXkFtZTcwNzc1MDAwNw@@._V1_UY209_CR4,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTUwMjkxMTI5Ml5BMl5BanBnXkFtZTcwNTU0NDAwNA@@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMjUyODkwODUyMF5BMl5BanBnXkFtZTcwMzU3MjYxMw@@._V1_UY209_CR22,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTczMjUwNDUzMF5BMl5BanBnXkFtZTgwNDA4OTAzMTE@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTUxNzY3NzYwOV5BMl5BanBnXkFtZTgwNzQ3Mzc4MTI@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMjExNTM5NDE4NV5BMl5BanBnXk

https://m.media-amazon.com/images/M/MV5BOGY2MjI5MDQtOThmMC00ZGIwLWFmYjgtYWU4MzcxOGEwMGVkXkEyXkFqcGdeQXVyMzM4MjM0Nzg@._V1_UY209_CR116,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMzczMmYzYzAtNDJhYy00ZDM1LWE5NzgtMTVlNGU3YWMxNWMxXkEyXkFqcGdeQXVyNTQ2NTIzNQ@@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTU1ODAzMDk3Ml5BMl5BanBnXkFtZTYwOTEzMzE0._V1_UY209_CR1,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BNzRhODBhZTMtNTJkOS00Y2UxLWIzZjAtMTcyN2QyY2JhNjU1XkEyXkFqcGdeQXVyMTIxMTA0Ng@@._V1_UY209_CR16,0,140,209_AL_.jpg
9
https://m.media-amazon.com/images/M/MV5BMTg0MjI4NjY2M15BMl5BanBnXkFtZTcwOTgwNjQzMg@@._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTMyOTYzODQ5MF5BMl5BanBnXkFtZTcwMjE3MDgzMQ@@._V1_UY209_CR13,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMjE5NjE5Mzk2MV5BMl5BanBnXkFtZTcwODI5MDE1Ng@@._V1_UY209_CR4,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BNzMwODE1NjA3OV5BMl5BanBnXkFtZTgwNTY5MzM2OTE

In [35]:
A = []

for el in actors_array:
    A.append(el.descriptor)


In [36]:
tree = KDTree(A)

In [38]:
print(tree.query(actors_array[0].descriptor, k=10))

(array([0.        , 0.56276478, 0.58361897, 0.58502385, 0.59414782,
       0.5973952 , 0.60264409, 0.61793654, 0.62051688, 0.62598033]), array([  0, 403, 238,  53, 412,  40,  75,  92, 190, 214]))


In [40]:
print(actors_array[238].name)

 Kelsey Asbille

